In [5]:
%pip install -q cvxpy

  Obtaining dependency information for cvxpy from https://files.pythonhosted.org/packages/c2/8d/57a601819a720b4999017ad614f16fa87a0b8a53d0d28ede273d390f30e9/cvxpy-1.3.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for osqp>=0.4.1 from https://files.pythonhosted.org/packages/23/46/232041be3d85624ec4c20b3bdb8bd4bfb6640851901db155917e0c412aef/osqp-0.6.3-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/72.0 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/72.0 kB ? eta -:--:--
     ---------------- --------------------- 30.7/72.0 kB 325.1 kB/s eta 0:00:01
     -------------------------------- ----- 61.4/72.0 kB 465.5 kB/s eta 0:00:01
     -------------------------------------  71.7/72.0 kB 558.5 kB/s eta 0:00:01
     -------------------------------------- 72.0/72.0 kB 359.3 kB/s eta 0:00:00
     ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/8

In [2]:
import numpy as np
import cvxpy as cp

# Задача 1

In [3]:
# Исходные данные
D1_data = np.array([
    [150, 500, 0],
    [100, 0, 400],
    [50, 50, 0],
    [0, 100, 100],
    [100, 0, 50]
])
D3_data = np.array([
    [50, 50, 10, 300],
    [70, 150, 200, 250]
])
C1_data = np.array([
    [1, 2, 0],
    [2, 0, 1],
    [3, 3, 0],
    [0, 3, 2],
    [2, 0, 3]
])
C2_data = np.array([
    [2, 3],
    [1, 4],
    [3, 1]
])
C3_data = np.array([
    [2, 1, 3, 4],
    [3, 2, 3, 1]
])

power_data = np.array([600, 500, 700, 400, 600])
price_data = np.array([1.3, 1.5, 1.2, 1.7, 1.1])
orders_data = np.array([100, 200, 200, 500])
capacity_data = np.array([200, 500, 400])

In [4]:
# Переменные решения
I = cp.Variable(5, boolean=True)

D1_v = cp.Variable(C1_data.shape, nonneg=True)
D2_v = cp.Variable(C2_data.shape, nonneg=True)
D3_v = cp.Variable(C3_data.shape, nonneg=True)

In [5]:
# Ограничения
constraints = [
    cp.sum(D1_v, axis=1) <= cp.multiply(power_data, I),
    cp.sum(I) <= 2,

    cp.sum(D1_v, axis=0) <= capacity_data,
    D1_v <= D1_data,

    cp.sum(D1_v, axis=0) == cp.sum(D2_v, axis=1),
    cp.sum(D2_v, axis=0) == cp.sum(D3_v, axis=1),

    D3_v <= D3_data,
    cp.sum(D3_v, axis=0) == orders_data
]

In [6]:
# Целевая функция
obj = cp.Minimize(
    cp.sum(cp.multiply(cp.sum(D1_v, axis=1), price_data)) +
    cp.sum(cp.multiply(D1_v, C1_data)) +
    cp.sum(cp.multiply(D2_v, C2_data)) +
    cp.sum(cp.multiply(D3_v, C3_data))
)

In [7]:
# Решение задачи
prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value:", prob.value)
print("optimal var:\n", D1_v.value, "\n\n", D2_v.value, "\n\n", D3_v.value, "\n\n", I.value)

Restricted license - for non-production use only - expires 2025-11-24
status: optimal
optimal value: 6810.0
optimal var:
 [[150. 400.   0.]
 [ 50.   0. 400.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]] 

 [[  0. 200.]
 [400.   0.]
 [  0. 400.]] 

 [[ 50.  50.  10. 290.]
 [ 50. 150. 190. 210.]] 

 [1. 1. 0. 0. 0.]


In [9]:
# Дополнительные ограничения
capacity_buf = np.array([100, 0, 0])

constraints = [
    cp.sum(D1_v, axis=1) <= cp.multiply(power_data, I),
    cp.sum(I) <= 2,

    cp.sum(D1_v, axis=0) <= capacity_data - capacity_buf,
    D1_v <= D1_data,

    cp.sum(D1_v, axis=0) + capacity_buf == cp.sum(D2_v, axis=1),
    cp.sum(D2_v, axis=0) == cp.sum(D3_v, axis=1),

    D3_v <= D3_data,
    cp.sum(D3_v, axis=0) == orders_data
]

In [10]:
# Обновление целевой функции и решение
obj = cp.Minimize(
    cp.sum(cp.multiply(cp.sum(D1_v, axis=1), price_data)) +
    cp.sum(cp.multiply(D1_v, C1_data)) +
    cp.sum(cp.multiply(D2_v, C2_data)) +
    cp.sum(cp.multiply(D3_v, C3_data))
)

In [11]:
prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value:", prob.value)
print("optimal var:\n", D1_v.value, "\n\n", D2_v.value, "\n\n", D3_v.value, "\n\n", I.value)

status: optimal
optimal value: 6520.0
optimal var:
 [[100. 400.   0.]
 [  0.   0. 400.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]] 

 [[  0. 200.]
 [400.   0.]
 [  0. 400.]] 

 [[ 50.  50.  10. 290.]
 [ 50. 150. 190. 210.]] 

 [1. 1. 0. 0. 0.]


In [12]:
# Дополнительные переменные и ограничения
prod_cost = np.array([1, 2, 3, 2.5, 1.5])

constraints = [
    cp.sum(D1_v, axis=1) <= cp.multiply(power_data, I),
    cp.sum(I) <= 2,

    cp.sum(D1_v, axis=0) <= capacity_data,
    D1_v <= D1_data,

    cp.sum(D1_v, axis=0) == cp.sum(D2_v, axis=1),
    cp.sum(D2_v, axis=0) == cp.sum(D3_v, axis=1),

    D3_v <= D3_data,
    cp.sum(D3_v, axis=0) == orders_data
]

In [13]:
# Обновление целевой функции и решение
obj = cp.Minimize(
    cp.sum(cp.multiply(I, prod_cost)) +
    cp.sum(cp.multiply(cp.sum(D1_v, axis=1), price_data)) +
    cp.sum(cp.multiply(D1_v, C1_data)) +
    cp.sum(cp.multiply(D2_v, C2_data)) +
    cp.sum(cp.multiply(D3_v, C3_data))
)

In [14]:
prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value:", prob.value)
print("optimal var:\n", D1_v.value, "\n\n", D2_v.value, "\n\n", D3_v.value, "\n\n", I.value)

status: optimal
optimal value: 6813.0
optimal var:
 [[150. 400.   0.]
 [ 50.   0. 400.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]] 

 [[  0. 200.]
 [400.   0.]
 [  0. 400.]] 

 [[ 50.  50.  10. 290.]
 [ 50. 150. 190. 210.]] 

 [1. 1. 0. 0. 0.]


In [15]:
# Еще больше переменных и ограничений
capacity_cost = np.array([2, 1.5, 3])

constraints = [
    cp.sum(D1_v, axis=1) <= cp.multiply(power_data, I),
    cp.sum(I) <= 2,

    cp.sum(D1_v, axis=0) <= capacity_data,
    D1_v <= D1_data,

    cp.sum(D1_v, axis=0) == cp.sum(D2_v, axis=1),
    cp.sum(D2_v, axis=0) == cp.sum(D3_v, axis=1),

    D3_v <= D3_data,
    cp.sum(D3_v, axis=0) == orders_data
]

In [16]:
# Обновление целевой функции и решение
obj = cp.Minimize(
    cp.sum(cp.multiply(cp.sum(D1_v, axis=0), capacity_cost)) +
    cp.sum(cp.multiply(cp.sum(D1_v, axis=1), price_data)) +
    cp.sum(cp.multiply(D1_v, C1_data)) +
    cp.sum(cp.multiply(D2_v, C2_data)) +
    cp.sum(cp.multiply(D3_v, C3_data))
)

In [18]:
prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value:", prob.value)
print("optimal var:\n", D1_v.value, "\n\n", D2_v.value, "\n\n", D3_v.value, "\n\n", I.value)

status: optimal
optimal value: 9010.0
optimal var:
 [[150. 400.   0.]
 [ 50.   0. 400.]
 [  0.   0.   0.]
 [  0.   0.   0.]
 [  0.   0.   0.]] 

 [[  0. 200.]
 [400.   0.]
 [  0. 400.]] 

 [[ 50.  50.  10. 290.]
 [ 50. 150. 190. 210.]] 

 [1. 1. 0. 0. 0.]


# Задача 3

In [19]:
X = cp.Variable(2, integer=True)
Y = cp.Variable(3, integer=True)
Z = cp.Variable(3, integer=True)
K = cp.Variable(integer=True)

In [20]:
constraints = [
    X >= 0, Y >= 0, Z >= 0, K >= 0,

    cp.sum(X) <= 100, cp.sum(Y) <= 200, cp.sum(Z) <= 300,

    2 * X[0] + X[1] + 3 * Y[0] + Y[1] + 4 * Z[0] + 2 * Z[1] + Z[2] >= 3 * K,
    X[1] + Y[1] + 2 * Y[2] + Z[1] + 2 * Z[2] >= 2 * K
]

In [21]:
obj = cp.Maximize(K)

In [22]:
prob = cp.Problem(obj, constraints)
prob.solve()
print("status:", prob.status)
print("optimal value:", prob.value)
print("optimal var:\n", X.value, "\n\n", Y.value, "\n\n", Z.value)
print()

status: optimal
optimal value: 341.0
optimal var:
 [  0. 100.] 

 [200.   0.   0.] 

 [  9.   0. 291.]

